# Post 레코드 생성

In [1]:
from blog.models import Post

In [19]:
p = Post(title="오늘 점심 메뉴", text="머지?")

In [20]:
p.save() # 이게 안되는 이유는 author_id를 할당 받지 못해서!

IntegrityError: NOT NULL constraint failed: blog_post.author_id

In [21]:
# 위의 안되는 문제를 해결하기 위해서 user테이블을 가져와야함
# 밑의 패키지를 맨 첫 셀에 추가해야함
from django.contrib.auth.models import User

In [22]:
# User.objects.all()  # system에서 만든 user테이블임

u = User.objects.all().get(username='lee')
print(u)

p = Post(title="오늘 점심 메뉴", text="머지?", author=u)
p.save()

lee


### DB browser for sqlite3의 데이터 보기에서 blog_post테이블에 가면 오늘 점심 메뉴 라는 타이틀의 레코드가 새로 생성된 것을 알 수 있음

In [23]:
p.title="오늘 점심 메뉴~~~~~"
p.publish()

In [24]:
print(p.title)

오늘 점심 메뉴~~~~~


In [25]:
Post.objects.create(title="오늘 저녁 메뉴", text="머지?", author=u)

<Post: 오늘 저녁 메뉴>

---

- mysite>blog>views.py를 다음과 같이 기입한다.

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse

# Create your views here.


def index(request):
    return HttpResponse("ok")

def index2(request, name):
    return HttpResponse)

- mysite>blog 디렉토리에 urls.py라는 파일을 만들어서 다음 셀과 같이 입력한다.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index),
    path('<name>/', views.index2),    # <name>은 parameter를 의미함
]

mysite>mysite 디렉토리에서 urls.py에서 path를 하나 더 추가한다.
    - path("blog/", include("blog.urls"))

- 127:0.0.1:8000/blog 페이지와 127:0.0.1:8000/blog/abc 페이지를 비교해보자.
    - 전자는 index함수를, 후자는 index2함수를 쓴 것임을 알 수 있다.

- mysite>blog>urls.py에서 path 하나를 더 추가하자.
    - `path('<int:pk>/detail', views.index3)`, # pk가 정수값으로 주어진다고 가정
 
 
-  그리고 mysite>blog>views.py에서 index3함수를 다음과 같이 추가하자

In [ ]:
def index3(request, pk) :
    return HttpResponse("ok" + str(pk))

또 127.0.0.1:8000/blog/123 페이지로 이동하며 결과가 보인다.

---

- mysite>blog>views.py를 다음과 같이 전체 수정하자

In [ ]:
from django.shortcuts import render , get_object_or_404
from django.http import HttpResponse
from blog.models import Post

# Create your views here.


def index(request):
    return HttpResponse("ok")

def index2(request, name):
    return HttpResponse("ok" + name)

def index3(request, pk) :
    p = Post.objects.get(pk=pk)    # blog_post 테이블에서 id값을 가져옴
    #p = get_object_or_404(Post, pk=pk)
    return HttpResponse("ok" + p.title)   # 해당 id의 title값을 가져와서 출력함

- 127.0.0.1:8000/blog/#/detail 페이지로 가서 결과를 확인해보자
    - 여기서 #은 DB browser에서 blog_post 테이블에 있는 레코드들의 id값(숫자)들을 넣어보면서 관찰해보자

---

- mysite>blog>views.py에 다음 list함수를 추가한다.

In [ ]:
def list(request):
    data = Post.objects.all()
    return render(request, 'blog/list.html', {"data": data})

- mysite>blog>urls.py 파일에 path하나 추가한다.
    - path('list', views.list),

In [ ]:
My blog <br>

{% for d in data %}
    {{d.title}} <br>
    {{d.text}}
    <hr>

{% endfor%}

## base template, sub template
- copy right 혹은 배너(base template) 등이 바뀌지 않고 부분(sub template)만 바뀌게 하는 것을 처리하기 위해서 template를 나눈다.

- mysite>templates 디렉토리에서 blog 폴더를 새로 만들고 그 안에 list.html이라는 파일을 하나 만들어서 다음과 같이 입력한다.

In [ ]:
{% extends 'blog/base.html' %}   <!-- 파이썬에서 import 하는 개념이라고 생각하면 됌-->

{% block content %}

{% for d in data %}
<a href="{{d.pk}}/detail">{{d.title}} </a><br>
{% endfor%}


{% endblock %}

- mysite>templates>blog 디렉토리에서 base.html이라는 파일을 만들과 다음과 같이 입력한다.

In [ ]:
<font color="red"><h1>My blog</h1></font>

{% block content %}
{% endblock %}


<br><br>
copy right....<br>
서울특별시.....

- mysite>templates>blog 디렉토리에서 detail.html 파일을 만들고 다음과 같이 입력하다.

In [ ]:
{% extends 'blog/base.html' %}
{% block content %}

    aaa
    {{d.text|linebreaks}}    <!--'|linebreaks' : enter를 인식 못해서 이걸 자동적으로 <br>로 바꿔줌-->


{% endblock %}




- mysite>blog>urls.py도 다음과 같이 바꿔준다.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index),
    path('<name>/', views.index2),    # <name>은 parameter를 의미함
    # path('<int:pk>/detail', views.index3),    # pk가 정수값으로 주어진다고 가정
    path('list', views.list),
    path('<int:pk>/detail', views.detail),
]

- 127.0.0.1:8000/blog/list에서 확인 ㄱㄱ 

---

- 아랫줄에 있는 패키지를 mysite>blog>views.py의 상단에 추가하고 다음 셀에 있는 class를 추가한다.

In [ ]:
# 추가할 패키지




# view를 상속받기 # get으로 할지 post로 할지 나눠서 class를 정의한다.
class PostView(View):
    def get(self):
        return HttpResponse("get ok~")
    def post(self):
        return HttpResponse("post ok~")

- mysite>blog>urls.py에 path추가한다.
    - path('list2', views.PostView.as_view(), name=login),

- mysite>blog>views.py에 class 하나 더 추가하고 패키지도 더 추가해야 하는데... 그냥 다음 셀 복사해서 views.py에 전체 다 붙여넣기 ㄱㄱ

In [ ]:
from django.shortcuts import render , get_object_or_404, redirect
from django.http import HttpResponse
from blog.models import Post
from django.views.generic import View

# Create your views here.


def index(request):
    return HttpResponse("ok")

def index2(request, name):
    return HttpResponse("ok" + name)

# def index3(request, pk) :
#     p = Post.objects.get(pk=pk)
#     #p = get_object_or_404(Post, pk=pk)
#     return HttpResponse("ok" + p.title)

def list(request):
    data = Post.objects.all()
    return render(request, 'blog/list.html', {"data": data})

def detail(request, pk):
    p = get_object_or_404(Post, pk=pk)
    return render(request, 'blog/detail.html', {"d": p})


# view를 상속받기 # get으로 할지 post로 할지 나눠서 class를 정의한다.
class PostView(View):
    def get(self):
        return HttpResponse("get ok~")
    def post(self):
        return HttpResponse("post ok~")

class LoginView(View, request):
    def get(self, request):
        return render(request, "blog/login.html")
    def post(self):
        return redirect("list")

- mysite>templates>blog에 login.html 파일을 만들고 다음 셀의 내용으로 채워넣기

In [ ]:
<form action="{% url 'login' %}" method=post>
    {% csrf_token %}
    username <input type=text name=username> <br>
    password <input type=password name=password> <br>
    <input type=submit vlaue="로그인">
</form>

- mysite>blog>urls.py에서 path하나 더 추가한다.
    - path('login', views.LoginView.as_view()),

## Password를 암호화하기

In [29]:
# Password를 암호화하기
from django.contrib.auth import authenticate

user = authenticate(username="home", password="choikt1234a")
if user == None : print(user)

None


- 위의 내용을 이용해서 mysite>blog>views.py에 추가한다.
    - 패키지는 맨 상단에 (from django.contrib.auth import authenticate)
    - 내용은 새로운 함수를 만들어서 LoginView class 안에 넣는다.(다음 셀의 내용을 PostView 클래스에 수정해서 넣는다.)

In [ ]:
class LoginView(View):
    def get(self, request):
        return render(request, "blog/login.html")
    def post(self, request):
        username = request.POST.get("username")
        password = request.POST.get("password")
        user = authenticate(username=username, password=password)
        if user == None:
            return redirect("login")
        return redirect("list")